# Create Slice for Fabric
This is the slice creation for the ```ECE 4400/6400 final project```. This creates a FABRIC slice.
 

## 1. Set up the Experiment

In this section, you will use the Fablib manager to create a new slice that is composed of 3 nodes for the Exploring Queues Assignment.
### 1.1 Reserve Resources
In the 'EDUKY' site, we will reserve a set of 3 nodes arranged in a star topology, with one node designated as a server. Each node will have the following specifications: 1 CPU core, 2GB of RAM, and 10GB of storage capacity. The server node will have 3 network cards (NICs), while each client node will have 1 NIC. All nodes will be preloaded with an 'Ubuntu' Linux OS. The server node will be responsible for passing traffic from the server to the client nodes.

In [1]:
# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
fablib.show_config()
import json
import traceback

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,0508639f-d515-44c8-b922-cf4cf000b6d9
Bastion Host,bastion.fabric-testbed.net
Bastion Username,jysilva_0000293756
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key


### 1.2 Set up the Experiment Network
The network configuration part of this slice will connect the nodes with specific IPs such that the nodes are able to communicate with each other. The slice will use the network name to find the corresponding devices (NIC cards) used in the slice creation so that we can provide the correct connections in the network, in this case, "Lan1" and "Lan2".

This slice contains 2 subnetworks in IPv4: 10.1.1.0/24 (between client1 and the server, "Lan1") & 11.1.1.0/24 (between client2 and the server, "Lan2"). This slice will provide IP 10.1.1.2 and 11.1.1.2 following the guidelines set by the network subnet. It will connect 10.1.1.2 to the card connected to client1, and 11.1.1.2 to client2.

After providing the IPs to the corresponding devices, the cell will verify that all interfaces are up and execute "ip a" to output the connections that are active. Then, we will add the additional routes so that the nodes are aware of the active connections using the route command.

In [ ]:
# Setup Network
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:    
    client1 = slice.get_node(name="client1") 
    client2 = slice.get_node(name="client2")
    client3 = slice.get_node(name="client3")
    router1 = slice.get_node(name="router1") 
    router2 = slice.get_node(name="router2")
    router3 = slice.get_node(name="router3")
    server = slice.get_node(name="server")
    
    subnet1 = IPv4Network("10.1.1.0/24")
    subnet2 = IPv4Network("11.1.1.0/24")
    subnet3 = IPv4Network("12.1.1.0/24")
    subnet4 = IPv4Network("13.1.1.0/24") # ISP subnet

    # Client 1 connections
    client1_iface = client1.get_interface(network_name="Lan1")
    client1_iface.ip_addr_add(addr="10.1.1.1", subnet=subnet1)

    router1_iface = router1.get_interface(network_name="Lan1")
    router1_iface.ip_addr_add(addr="10.1.1.2", subnet=subnet1)

    router1_iface2 = router1.get_interface(network_name="Lan4")
    router1_iface2.ip_addr_add(addr="13.1.1.2", subnet=subnet4)
    
    server_iface = server.get_interface(network_name="Lan4")
    server_iface.ip_addr_add(addr="13.1.1.1", subnet=subnet4)

    # Cient 2 connections
    client2_iface = client2.get_interface(network_name="Lan2")
    client2_iface.ip_addr_add(addr="11.1.1.1", subnet=subnet2)

    router2_iface = router2.get_interface(network_name="Lan2")
    router2_iface.ip_addr_add(addr="11.1.1.2", subnet=subnet2)

    router2_iface2 = router2.get_interface(network_name="Lan4")
    router2_iface2.ip_addr_add(addr="13.1.1.4", subnet=subnet4)
    
    server_iface2 = server.get_interface(network_name="Lan4")
    server_iface2.ip_addr_add(addr="13.1.1.3", subnet=subnet4)

    # Client 3 Connections
    client3_iface = client3.get_interface(network_name="Lan3")
    client3_iface.ip_addr_add(addr="12.1.1.1", subnet=subnet3)

    router3_iface = router3.get_interface(network_name="Lan3")
    router3_iface.ip_addr_add(addr="12.1.1.2", subnet=subnet3)

    router3_iface2 = router3.get_interface(network_name="Lan4")
    router3_iface2.ip_addr_add(addr="13.1.1.6", subnet=subnet4)
    
    server_iface3 = server.get_interface(network_name="Lan4")
    server_iface3.ip_addr_add(addr="13.1.1.5", subnet=subnet4)
    
    #turn on interfaces
    for node in slice.get_nodes():
        for interface in node.get_interfaces():
            stdout, stderr = node.execute(f'sudo ip link set dev { interface.get_device_name()} up')
        print(f"Network status at node {node.get_name()}")
        node.execute("ip a")
    
except Exception as e:
    print(f"Exception: {e}")


### 1.3 Configure the software needed for the nodes in the experiment
In this section of the slice creation, we will add any additional software, tools or scripts that we need for our experiments. For this slice we only need to install "iperf", we provide the software by running node.execute() and letting the node know to download the software.

In [ ]:
# Install Software
for node in slice.get_nodes():
    node.execute("sudo apt update;sudo apt -y install iperf;sudo apt -y install net-tools;sudo apt -y install iperf3; sudo apt -y install ffmpeg; sudo snap install yt-dlp")

### 1.4 Set up the IP routes and IP forwarding for the subnets
In this section of the slice creation, we add the IP routing & forwarding necessary for the subnets to communicate. Add a way for clients to communicate back to the server, and a way for the server to communicate to the clients (Both through the routers) 

Keep in mind the output of 1.2 for setting up Port Forwarding & for later testing with iperf. enp7s0 & enp8s0 tend to flip between which is used for input from server and output to client. Uncomment/Comment on "Allow forwarding through routers" based on 1.2 Output.

In [ ]:
#Handle IP routes & IP forwarding
client1.execute("sudo ip route add 13.1.1.0/24 via 10.1.1.2")
client2.execute("sudo ip route add 13.1.1.0/24 via 11.1.1.2") 
client3.execute("sudo ip route add 13.1.1.0/24 via 12.1.1.2") 

#server.execute("sudo sysctl -w net.ipv4.ip_forward=1")
server.execute("sudo ip route add 10.1.1.0/24 via 13.1.1.2")
server.execute("sudo ip route add 11.1.1.0/24 via 13.1.1.4")
server.execute("sudo ip route add 12.1.1.0/24 via 13.1.1.6") 

router1.execute("sudo sysctl -w net.ipv4.ip_forward=1") 
router2.execute("sudo sysctl -w net.ipv4.ip_forward=1")
router3.execute("sudo sysctl -w net.ipv4.ip_forward=1")  

#Allow forwarding through routers 

router1.execute("sudo iptables -A FORWARD -i enp7s0 -o enp8s0 -j ACCEPT") 
#router1.execute("sudo iptables -A FORWARD -i enp8s0 -o enp7s0 -j ACCEPT")
 
#router2.execute("sudo iptables -A FORWARD -i enp7s0 -o enp8s0 -j ACCEPT") 
router2.execute("sudo iptables -A FORWARD -i enp8s0 -o enp7s0 -j ACCEPT")

#router3.execute("sudo iptables -A FORWARD -i enp7s0 -o enp8s0 -j ACCEPT") 
router3.execute("sudo iptables -A FORWARD -i enp8s0 -o enp7s0 -j ACCEPT") 

#Install caca on clients 
client1.execute("sudo apt install ffmpeg libcaca0") 
client2.execute("sudo apt install ffmpeg libcaca0") 
client3.execute("sudo apt install ffmpeg libcaca0")  

## Continue to The experiment notebook

Once You have completed this notebook you should be able to continue.